In [28]:
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from tkinter import *
from PIL import ImageTk, Image

import nbimporter
#import NewModel as NM

master = Tk()

# Read the Image
image = Image.open("yelp.png")
 
# Resize the image using resize() method
resize_image = image.resize((100, 100))
 
img = ImageTk.PhotoImage(resize_image)
 
# create label and add resize image
label1 = Label(image=img)
label1.image = img
label1.grid(row=0)

Label(master, text='Enter Review: ').grid(row=1)
e1 = Entry(master)
e1.grid(row=2, column=0)

def buttonClick():
    
    #model = NM.model()
    
    review = e1.get()
        
    input_sentence = review
    encoded_input = tokenizer.encode(input_sentence, add_special_tokens=False, return_tensors='pt')

    #Feed input to BERT model and get output token embeddings
    with torch.no_grad():
        output = model(encoded_input)[0]
        
    print(model.predict(output))

    print(output)
    print(output.shape)
    print(review)
    

button = Button(master, text="Enter", command=buttonClick)
button.grid(row=3, column=0)
        
master.mainloop()